In [1]:
import numpy as np
import pandas as pd
import os
import re
import warnings
from sklearn.base import clone
from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
from sklearn.datasets import make_classification
from scipy.optimize import minimize

from IPython.display import clear_output
import warnings
from tqdm import tqdm
from colorama import Fore, Style


from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor, RandomForestRegressor, GradientBoostingRegressor

In [2]:
SEED = 42
n_splits = 5

warnings.filterwarnings('ignore')
pd.options.display.max_columns = None

In [3]:
import random
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    # torch.manual_seed(seed)
    # torch.cuda.manual_seed(seed)
    # torch.backends.cudnn.deterministic = True
    # torch.backends.cudnn.benchmark = True
seed_everything(2024)

### Data Processing

In [4]:
train  = pd.read_csv('./train.csv')
test   = pd.read_csv('./test.csv')
train = train.dropna(subset=["sii"]).reset_index(drop=True)

In [5]:
def feature_engineering(df):
    # season_zone_cols = [col for col in df.columns if 'Season' in col or 'Zone' in col]
    
    # df = df.drop(season_zone_cols, axis=1) 
    LBS_TO_KG = 0.453592
    IN_TO_CM = 2.54
    
    df['PAQ_Total'] = df['PAQ_C-PAQ_C_Total'].combine_first(df['PAQ_A-PAQ_A_Total'])
    df['BIA-BIA_DEE'] = df['BIA-BIA_DEE'] / df['Physical-Weight']
    df['BMI'] = df['Physical-BMI'].combine_first(df['BIA-BIA_BMI'])
    df['BMR'] = df.apply(
        lambda row: (
            88.362 + (13.397 * (row['Physical-Weight'] * LBS_TO_KG)) +
            (4.799 * (row['Physical-Height'] * IN_TO_CM)) - 5.677 * row['Basic_Demos-Age']
            if row['Basic_Demos-Sex'] == 0 else
            447.593 + (9.247 * (row['Physical-Weight'] * LBS_TO_KG)) +
            (3.098 * (row['Physical-Height'] * IN_TO_CM)) - 4.330 * row['Basic_Demos-Age']
        ),
            axis=1
        )
    df['BMR'] = df['BMR'].combine_first(df['BIA-BIA_BMR'])
    df['BIA-BIA_Activity_Level_num'] = df['BIA-BIA_Activity_Level_num'].apply(
    lambda x: 1 if x <= 1.5 else
              2 if x <= 2.5 else
              3 if x <= 3.5 else
              4 if x <= 4.5 else
              5
    )

    df.drop(columns=[
        'PAQ_A-PAQ_A_Total', 
        'PAQ_C-PAQ_C_Total', 
        'BIA-BIA_Fat',
        'BIA-BIA_BMI',
        'Physical-BMI',
        'BIA-BIA_BMR',
        'SDS-SDS_Total_T',
        'Physical-HeartRate',
        'PAQ_Total',
        'BIA-BIA_LST',
        # 'Physical-Season',
        # 'FGC-FGC_GSND_Zone'
        # 'PreInt_EduHx-computerinternet_hoursday'
        # 'CGAS-CGAS_Score'
    ], inplace=True)
    
    return df

In [6]:
drop_cols = list(set(train.columns) - set(test.columns))

In [7]:
X_train = train.drop(["id"] + drop_cols, axis=1)
X_test = test.drop(["id"], axis=1)
X_train = feature_engineering(X_train)
X_test = feature_engineering(X_test)
y = train["sii"].astype(int)

In [8]:
# if np.any(np.isinf(X_train)):
#     X_train = X_train.replace([np.inf, -np.inf], np.nan)

### One-hot encoding

In [9]:
category_cols = list(X_train.columns[X_train.dtypes=="object"])
category_cols

['Basic_Demos-Enroll_Season',
 'CGAS-Season',
 'Physical-Season',
 'Fitness_Endurance-Season',
 'FGC-Season',
 'BIA-Season',
 'PAQ_A-Season',
 'PAQ_C-Season',
 'SDS-Season',
 'PreInt_EduHx-Season']

In [10]:
def update(df):
    global category_cols
    for c in category_cols: 
        df[c] = df[c].fillna('Missing')
        df[c] = df[c].astype('category')
    return df

def create_mapping(column, train, test):
    x1 = list(X_train[col])
    x2 = list(X_test[col])
    x1.extend(x2)
    unique_values = pd.Series(x1).unique()
    return {value: idx for idx, value in enumerate(unique_values)}

X_train = update(X_train)
X_test = update(X_test)
for col in category_cols:
    mapping = create_mapping(col, X_train, X_test)
    
    X_train["enc_"+col] = X_train[col].replace(mapping).astype(int)
    X_test["enc_"+col] = X_test[col].replace(mapping).astype(int)

In [11]:
X_train = X_train.drop(category_cols, axis=1)
X_test = X_test.drop(category_cols, axis=1)

In [12]:
assert len(X_train.columns) == len(X_test.columns)

### Voting Regressor 

In [13]:
def quadratic_weighted_kappa(y_true, y_pred):
    return cohen_kappa_score(y_true, y_pred, weights='quadratic')

def threshold_Rounder(oof_non_rounded, thresholds):
    return np.where(oof_non_rounded < thresholds[0], 0,
                    np.where(oof_non_rounded < thresholds[1], 1,
                             np.where(oof_non_rounded < thresholds[2], 2, 3)))

def evaluate_predictions(thresholds, y_true, oof_non_rounded):
    rounded_p = threshold_Rounder(oof_non_rounded, thresholds)
    return -quadratic_weighted_kappa(y_true, rounded_p)

In [14]:
def TrainML(model_class):
    global X_train, X_test, y, test
    SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
    
    train_S = []
    test_S = []
    
    oof_non_rounded = np.zeros(len(y), dtype=float) 
    oof_rounded = np.zeros(len(y), dtype=int) 
    test_preds = np.zeros((len(X_test), n_splits))

    for fold, (train_idx, val_idx) in enumerate(tqdm(SKF.split(X_train, y), desc="Training Folds", total=n_splits)):
        x_train, x_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model = clone(model_class)
        model.fit(x_train, y_train)

        y_train_pred = model.predict(x_train)
        y_val_pred = model.predict(x_val)

        oof_non_rounded[val_idx] = y_val_pred
        y_val_pred_rounded = y_val_pred.round(0).astype(int)
        oof_rounded[val_idx] = y_val_pred_rounded

        train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
        val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)

        train_S.append(train_kappa)
        test_S.append(val_kappa)
        
        test_preds[:, fold] = model.predict(X_test)
        
        print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
        clear_output(wait=True)

    print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
    print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")

    KappaOPtimizer = minimize(evaluate_predictions,
                              x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                              method='Nelder-Mead')
    assert KappaOPtimizer.success, "Optimization did not converge."
    
    oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
    tKappa = quadratic_weighted_kappa(y, oof_tuned)

    print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")

    tpm = test_preds.mean(axis=1)
    tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)
    
    submission = pd.DataFrame({
        'id': test['id'],
        'sii': tpTuned
    })

    return submission

In [15]:
# def TrainMLFeatureSelection(model_class, X_train_input, X_test_input, y_input, test_input, excluded_feature):
#     feature = ["Physical-Diastolic_BP","BIA-BIA_Activity_Level_num", "Physical-HeartRate","BIA-BIA_LST", "BMR","FGC-FGC_SRL", "BMI", "FGC-FGC_GSD_Zone","BIA-BIA_DEE","enc_BIA-Season","BIA-BIA_FMI", "FGC-FGC_GSD","BIA-BIA_SMM", "Fitness_Endurance-Max_Stage","FGC-FGC_SRL_Zone","BIA-BIA_Frame_num","Physical-Weight","enc_PAQ_C-Season","BIA-BIA_FFMI","FGC-FGC_GSND_Zone", "FGC-FGC_TL_Zone","BIA-BIA_ICW","BIA-BIA_BMC", "Fitness_Endurance-Time_Sec", "CGAS-CGAS_Score", "FGC-FGC_PU_Zone","FGC-FGC_CU","PAQ_Total", "FGC-FGC_SRR","BIA-BIA_LDM","Physical-Height","enc_CGAS-Season","enc_FGC-Season", "enc_Fitness_Endurance-Season","enc_PreInt_EduHx-Season","FGC-FGC_GSND","enc_SDS-Season", "FGC-FGC_TL", "FGC-FGC_CU_Zone","BIA-BIA_ECW", "Physical-Waist_Circumference", "FGC-FGC_SRR_Zone","enc_PAQ_A-Season","BIA-BIA_TBW", "Fitness_Endurance-Time_Mins","enc_Basic_Demos-Enroll_Season","Basic_Demos-Sex","BIA-BIA_FFM", "Physical-Systolic_BP","FGC-FGC_PU", "enc_Physical-Season","Basic_Demos-Age", "PreInt_EduHx-computerinternet_hoursday", "SDS-SDS_Total_Raw"]
#     score = [0.4655470176249047,0.46295027383650555,0.462676822633297,0.46089755998943405,0.4606992990565012,0.46055216946452127,0.46047234641815504,0.4602889112054913,0.4599243737655201,0.4592060548256611,0.45883931359063856,0.4588027962643718,0.45863583922807105,0.458597342409211,0.4585803644275225,0.4583744652846331,0.4581314641102292,0.4581295191433876,0.4579331062445555,0.45769648750713265,0.4576502518537059,0.4576462567444265,0.45700450131052694,0.45677504621535947,0.45665322119104634,0.4565952080706178,0.4564165132625905,0.45639459076854294,0.4562166286755662,0.456195696233764,0.4560189208181894,0.4559118593358582,0.45546666140536063,0.4553885687587603,0.4553733775796728,0.4552924772490142,0.45507320087177483,0.45439362394089766,0.4542710841330666,0.45412180052956763,0.45410026939409154,0.4538095381585636,0.4535580985193388,0.45345611138533815,0.4533610988177278,0.4530387233346388,0.452981932491742,0.45270953666630454,0.4499278396753007,0.44992423038613427,0.4491412553407236,0.44374618373202634,0.42552387351446685,0.41551223280103444]
#     return pd.DataFrame({'feature': feature, 'score': score})

In [16]:
def TrainMLFeatureSelection(model_class, X_train_input, X_test_input, y_input, test_input, excluded_feature):

    X_train = X_train_input
    X_test = X_test_input 
    y = y_input
    test = test_input

    all_columns = X_train.columns
    results = {'feature': [], 'score': []}

    for col in all_columns:
        X_train = X_train_input.drop([col], axis=1)
        X_test = X_test_input.drop([col], axis=1)
    
        SKF = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=SEED)
        
        train_S = []
        test_S = []
        
        oof_non_rounded = np.zeros(len(y), dtype=float) 
        oof_rounded = np.zeros(len(y), dtype=int) 
        test_preds = np.zeros((len(X_test), n_splits))
    
        for fold, (train_idx, val_idx) in enumerate(tqdm(SKF.split(X_train, y), desc=f"Training Folds {excluded_feature}", total=n_splits)):
            x_train, x_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
    
            model = clone(model_class)
            model.fit(x_train, y_train)
    
            y_train_pred = model.predict(x_train)
            y_val_pred = model.predict(x_val)
    
            oof_non_rounded[val_idx] = y_val_pred
            y_val_pred_rounded = y_val_pred.round(0).astype(int)
            oof_rounded[val_idx] = y_val_pred_rounded
    
            train_kappa = quadratic_weighted_kappa(y_train, y_train_pred.round(0).astype(int))
            val_kappa = quadratic_weighted_kappa(y_val, y_val_pred_rounded)
    
            train_S.append(train_kappa)
            test_S.append(val_kappa)
            
            test_preds[:, fold] = model.predict(X_test)
            
            print(f"Fold {fold+1} - Train QWK: {train_kappa:.4f}, Validation QWK: {val_kappa:.4f}")
            clear_output(wait=True)
    
        print(f"Mean Train QWK --> {np.mean(train_S):.4f}")
        print(f"Mean Validation QWK ---> {np.mean(test_S):.4f}")
    
        KappaOPtimizer = minimize(evaluate_predictions,
                                  x0=[0.5, 1.5, 2.5], args=(y, oof_non_rounded), 
                                  method='Nelder-Mead')
        assert KappaOPtimizer.success, "Optimization did not converge."
        
        oof_tuned = threshold_Rounder(oof_non_rounded, KappaOPtimizer.x)
        tKappa = quadratic_weighted_kappa(y, oof_tuned)
    
        print(f"----> || Optimized QWK SCORE :: {Fore.CYAN}{Style.BRIGHT} {tKappa:.3f}{Style.RESET_ALL}")
    
        tpm = test_preds.mean(axis=1)
        tpTuned = threshold_Rounder(tpm, KappaOPtimizer.x)

        results['feature'].append(col)
        results['score'].append(tKappa)

    results = pd.DataFrame(results)
    return results

In [17]:
Params = {
    'learning_rate': 0.046,
    'max_depth': 12,
    'num_leaves': 478,
    'min_data_in_leaf': 13,
    'feature_fraction': 0.893,
    'bagging_fraction': 0.784,
    'bagging_freq': 4,
    'lambda_l1': 10,  # Increased from 6.59
    'lambda_l2': 0.01,  # Increased from 2.68e-06
    'device': 'cpu',
}


XGB_Params = {
    'learning_rate': 0.05,
    'max_depth': 6,
    'n_estimators': 200,
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'reg_alpha': 1,  # Increased from 0.1
    'reg_lambda': 5,  # Increased from 1
    'random_state': SEED,
    'tree_method': 'gpu_hist'
}


CatBoost_Params = {
    'learning_rate': 0.05,
    'depth': 6,
    'iterations': 200,
    'random_seed': SEED,
    'verbose': 0,
    'l2_leaf_reg': 10,  # Increase this value
    'task_type': 'GPU',
}

In [18]:
Light = LGBMRegressor(**Params, random_state=SEED, verbose=-1, n_estimators=300)
XGB_Model = XGBRegressor(**XGB_Params)
CatBoost_Model = CatBoostRegressor(**CatBoost_Params)

# Combine models using Voting Regressor
voting_model = VotingRegressor(estimators=[
    ('lightgbm', Light),
    ('xgboost', XGB_Model),
    ('catboost', CatBoost_Model),
], weights=[4.0,4.0,5.0])

In [19]:
submission_df = TrainML(voting_model)
submission_df.to_csv("submission.csv", index=False)
submission_df


Training Folds: 100%|██████████| 5/5 [00:23<00:00,  4.67s/it]

Mean Train QWK --> 0.7306
Mean Validation QWK ---> 0.3916
----> || Optimized QWK SCORE ::  0.470


,id,sii
0,00008ff9,1
1,000fd460,0
2,00105258,0
3,00115b9f,0
4,0016bb22,1
5,001f3379,1
6,0038ba98,0
7,0068a485,0
8,0069fbed,1
9,0083e397,1


In [20]:
features_to_be_excluded = ["Physical-Diastolic_BP",
    "BIA-BIA_Activity_Level_num" ,
    # "Physical-HeartRate",
    # "BIA-BIA_LST" ,
    "BMR",
    "FGC-FGC_SRL" ,
    "BMI" ,
    "FGC-FGC_GSD_Zone",
    "BIA-BIA_DEE",
    "enc_BIA-Season",
    "BIA-BIA_FMI" ,
    "FGC-FGC_GSD",
    "BIA-BIA_SMM" ,
    "Fitness_Endurance-Max_Stage",
    "FGC-FGC_SRL_Zone",
    "BIA-BIA_Frame_num",
    "Physical-Weight",
    "enc_PAQ_C-Season",
    "BIA-BIA_FFMI",
    "FGC-FGC_GSND_Zone" ,
    "FGC-FGC_TL_Zone",
    "BIA-BIA_ICW",
    "BIA-BIA_BMC" ,
    "Fitness_Endurance-Time_Sec" ,
    "CGAS-CGAS_Score" ,
    "FGC-FGC_PU_Zone",
    "FGC-FGC_CU",
    # "PAQ_Total" ,
    "FGC-FGC_SRR",
    "BIA-BIA_LDM",
    "Physical-Height",
    "enc_CGAS-Season",
    "enc_FGC-Season" ,
    "enc_Fitness_Endurance-Season",
    "enc_PreInt_EduHx-Season",
    "FGC-FGC_GSND",
    "enc_SDS-Season" ,
    "FGC-FGC_TL" ,
    "FGC-FGC_CU_Zone",
    "BIA-BIA_ECW" ,
    "Physical-Waist_Circumference" ,
    "FGC-FGC_SRR_Zone",
    "enc_PAQ_A-Season",
    "BIA-BIA_TBW" ,
    "Fitness_Endurance-Time_Mins",
    "enc_Basic_Demos-Enroll_Season",
    "Basic_Demos-Sex",
    "BIA-BIA_FFM" ,
    "Physical-Systolic_BP",
    "FGC-FGC_PU" ,
    "enc_Physical-Season",
    "Basic_Demos-Age" ,
    "PreInt_EduHx-computerinternet_hoursday" ,
    "SDS-SDS_Total_Raw"]

In [21]:
from itertools import islice
from tqdm import tqdm
import pandas as pd
import numpy as np

excluding_rank_top10 = pd.DataFrame({'feature': [feat for feat in features_to_be_excluded]} )
excluding_rank_top15 = pd.DataFrame({'feature': [feat for feat in features_to_be_excluded]} )
excluding_rank_top20 = pd.DataFrame({'feature': [feat for feat in features_to_be_excluded]} )

excluding_rank_top10['top_10_times'] = 0
excluding_rank_top15['top_15_times'] = 0
excluding_rank_top20['top_20_times'] = 0

for i in range(10): 
    excluding_rank_top10[f'S{i+1} rank'] = 0
    excluding_rank_top15[f'S{i+1} rank'] = 0
    excluding_rank_top20[f'S{i+1} rank'] = 0


In [22]:
def backward_feature_selection():

    for i, excluded_feature in enumerate(tqdm(features_to_be_excluded[:10], desc="Processing Features")):
        X_train_input = X_train.drop([excluded_feature], axis=1)
        X_test_input = X_test.drop([excluded_feature], axis=1)
        
        results = TrainMLFeatureSelection(voting_model, X_train_input, X_test_input, y, test, excluded_feature)
        results = results.sort_values(by='score', ascending=False)
        results = results.reset_index(drop=True)
        for j in range(10):
            feat = results.loc[j, 'feature']
            excluding_rank_top10.loc[excluding_rank_top10['feature'] == feat, 'top_10_times'] += 1
            excluding_rank_top10.loc[excluding_rank_top10['feature'] == feat, f'S{i+1} rank'] = j + 1
        for j in range(15):
            feat = results.loc[j, 'feature']
            excluding_rank_top15.loc[excluding_rank_top10['feature'] == feat, 'top_15_times'] += 1
            excluding_rank_top15.loc[excluding_rank_top10['feature'] == feat, f'S{i+1} rank'] = j + 1
        for j in range(20):
            feat = results.loc[j, 'feature']
            excluding_rank_top20.loc[excluding_rank_top10['feature'] == feat, 'top_20_times'] += 1
            excluding_rank_top20.loc[excluding_rank_top10['feature'] == feat, f'S{i+1} rank'] = j + 1

In [23]:
# backward_feature_selection()

In [24]:
excluding_rank_top10 = excluding_rank_top10.sort_values(by='top_10_times', ascending=False)
excluding_rank_top15 = excluding_rank_top15.sort_values(by='top_15_times', ascending=False)
excluding_rank_top20 = excluding_rank_top20.sort_values(by='top_20_times', ascending=False)

columns_to_average = [f'S{i+1} rank' for i in range(10)]  # Create a list of column names
excluding_rank_top10['Avg rank'] = excluding_rank_top10[columns_to_average].apply(lambda row: row[row > 0].mean(), axis=1)
excluding_rank_top15['Avg rank'] = excluding_rank_top15[columns_to_average].apply(lambda row: row[row > 0].mean(), axis=1)
excluding_rank_top20['Avg rank'] = excluding_rank_top20[columns_to_average].apply(lambda row: row[row > 0].mean(), axis=1)



In [25]:
excluding_rank_top10.head(10)

,feature,top_10_times,S1 rank,S2 rank,S3 rank,S4 rank,S5 rank,S6 rank,S7 rank,S8 rank,S9 rank,S10 rank,Avg rank
0,Physical-Diastolic_BP,0,0,0,0,0,0,0,0,0,0,0,NaN
1,BIA-BIA_Activity_Level_num,0,0,0,0,0,0,0,0,0,0,0,NaN
2,BMR,0,0,0,0,0,0,0,0,0,0,0,NaN
3,FGC-FGC_SRL,0,0,0,0,0,0,0,0,0,0,0,NaN
4,BMI,0,0,0,0,0,0,0,0,0,0,0,NaN
5,FGC-FGC_GSD_Zone,0,0,0,0,0,0,0,0,0,0,0,NaN
6,BIA-BIA_DEE,0,0,0,0,0,0,0,0,0,0,0,NaN
7,enc_BIA-Season,0,0,0,0,0,0,0,0,0,0,0,NaN
8,BIA-BIA_FMI,0,0,0,0,0,0,0,0,0,0,0,NaN
9,FGC-FGC_GSD,0,0,0,0,0,0,0,0,0,0,0,NaN


In [26]:
excluding_rank_top15.head(15)

,feature,top_15_times,S1 rank,S2 rank,S3 rank,S4 rank,S5 rank,S6 rank,S7 rank,S8 rank,S9 rank,S10 rank,Avg rank
0,Physical-Diastolic_BP,0,0,0,0,0,0,0,0,0,0,0,NaN
1,BIA-BIA_Activity_Level_num,0,0,0,0,0,0,0,0,0,0,0,NaN
2,BMR,0,0,0,0,0,0,0,0,0,0,0,NaN
3,FGC-FGC_SRL,0,0,0,0,0,0,0,0,0,0,0,NaN
4,BMI,0,0,0,0,0,0,0,0,0,0,0,NaN
5,FGC-FGC_GSD_Zone,0,0,0,0,0,0,0,0,0,0,0,NaN
6,BIA-BIA_DEE,0,0,0,0,0,0,0,0,0,0,0,NaN
7,enc_BIA-Season,0,0,0,0,0,0,0,0,0,0,0,NaN
8,BIA-BIA_FMI,0,0,0,0,0,0,0,0,0,0,0,NaN
9,FGC-FGC_GSD,0,0,0,0,0,0,0,0,0,0,0,NaN


In [27]:
excluding_rank_top20.head(20)

,feature,top_20_times,S1 rank,S2 rank,S3 rank,S4 rank,S5 rank,S6 rank,S7 rank,S8 rank,S9 rank,S10 rank,Avg rank
0,Physical-Diastolic_BP,0,0,0,0,0,0,0,0,0,0,0,NaN
1,BIA-BIA_Activity_Level_num,0,0,0,0,0,0,0,0,0,0,0,NaN
2,BMR,0,0,0,0,0,0,0,0,0,0,0,NaN
3,FGC-FGC_SRL,0,0,0,0,0,0,0,0,0,0,0,NaN
4,BMI,0,0,0,0,0,0,0,0,0,0,0,NaN
5,FGC-FGC_GSD_Zone,0,0,0,0,0,0,0,0,0,0,0,NaN
6,BIA-BIA_DEE,0,0,0,0,0,0,0,0,0,0,0,NaN
7,enc_BIA-Season,0,0,0,0,0,0,0,0,0,0,0,NaN
8,BIA-BIA_FMI,0,0,0,0,0,0,0,0,0,0,0,NaN
9,FGC-FGC_GSD,0,0,0,0,0,0,0,0,0,0,0,NaN


In [28]:
excluding_rank_top10.to_csv("excluding_rank_top10.csv", index=False)
excluding_rank_top15.to_csv("excluding_rank_top15.csv", index=False)
excluding_rank_top20.to_csv("excluding_rank_top20.csv", index=False)
